In [1]:
import os
import glob
import numpy as np
import pandas as pd
import nibabel as nib
from nilearn import image, masking, input_data


In [4]:

denoised = 'denoised 6'

fwhm=0
tr = 2

tmp = 'Schaefer'
Schaefer_mask_file = '/Users/li/Desktop/template/Schaefer/tpl-MNI152NLin2009cAsym_res-02_atlas-Schaefer2018_desc-200Parcels7Networks_dseg.nii.gz'
roi_label_file = '/Users/li/Desktop/template/Schaefer/Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm.Centroid_RAS.csv'
Schaefer_mask = nib.load(Schaefer_mask_file)
roi_label = pd.read_csv(roi_label_file)['ROI Name'].values.tolist()

brain_mask_path = '/Users/li/Desktop/template/brain mask/tpl-MNI152NLin2009cAsym_res-02_desc-brain_mask.nii.gz'

sub_list = [f'sub-{x:0>3d}' for x in range(13,51)]
run_list = [1,2,3,4,5,6]

for sub in sub_list:
    new_folder_path = os.path.join('/Volumes/Li/task-debate/braindata', denoised, 'denoised', sub)
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)

    for run in run_list:
        run_file = f'/Volumes/Li/数据备份_task-debate/fmriprep/{sub}/func/{sub}_task-debate_run-{run}_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz'
        run_img = nib.load(run_file)
        
        confound_file = f'/Volumes/Li/数据备份_task-debate/fmriprep/{sub}/func/{sub}_task-debate_run-{run}_desc-confounds_timeseries.tsv'
        confound_df = pd.read_csv(confound_file, delimiter ='\t')
        
        # motion_outlier
        # motion_outlier_columns = [col for col in confound_df.columns if 'motion_outlier' in col]

        selected_confound_cols = [  # csf_wm	tcompcor	std_dvars	dvars	framewise_displacement	rmsd
            
                                    # 'global_signal', 'global_signal_derivative1', 'global_signal_power2', 'global_signal_derivative1_power2',
                                    # 'csf', 'csf_derivative1', 'csf_derivative1_power2', 'csf_power2',
                                    # 'white_matter', 'white_matter_derivative1', 'white_matter_derivative1_power2', 'white_matter_power2',

                                  'global_signal', 'csf', 'white_matter',
                                  'trans_x','trans_y','trans_z','rot_x','rot_y','rot_z',
                                  'trans_x_derivative1','trans_y_derivative1','trans_z_derivative1',
                                  'rot_x_derivative1','rot_y_derivative1','rot_z_derivative1'
                                  
                                  # 'trans_x_power2','trans_y_power2','trans_z_power2',
                                  # 'rot_x_power2','rot_y_power2','rot_z_power2',
                                  # 'trans_x_derivative1_power2','trans_y_derivative1_power2','trans_z_derivative1_power2',
                                  # 'rot_x_derivative1_power2','rot_y_derivative1_power2','rot_z_derivative1_power2'
                                ]
        # selected_confound_cols = selected_confound_cols + motion_outlier_columns
        
        confound_selected = confound_df[selected_confound_cols]
        confound_selected = confound_selected.fillna(value=0)
        confound_matrix = confound_selected.values

        denoised_img = image.clean_img(run_img, confounds=confound_matrix,
                                       detrend=True, standardize=True, high_pass=0.008, t_r=tr, mask_img=brain_mask_path)

        # smoothed_img = image.smooth_img(denoised_img, fwhm=fwhm)
        
        save_path = os.path.join('/Volumes/Li/task-debate/braindata', denoised, 'denoised', sub, f'{sub}_task-debate_run-{run}_denoised_smooth{fwhm}mm_space-MNI152NLin2009cAsym_res-2_bold.nii.gz')
        nib.save(denoised_img,save_path)
        # print(f'{sub} {run} denoised_smoothed data saved successfully!')

    print(f'{sub} Done!')
    print('-----------------------------------------------')



sub-013 Done!
-----------------------------------------------
sub-014 Done!
-----------------------------------------------
sub-015 Done!
-----------------------------------------------
sub-016 Done!
-----------------------------------------------
sub-017 Done!
-----------------------------------------------
sub-018 Done!
-----------------------------------------------
sub-019 Done!
-----------------------------------------------
sub-020 Done!
-----------------------------------------------
sub-021 Done!
-----------------------------------------------
sub-022 Done!
-----------------------------------------------
sub-023 Done!
-----------------------------------------------
sub-024 Done!
-----------------------------------------------
sub-025 Done!
-----------------------------------------------
sub-026 Done!
-----------------------------------------------
sub-027 Done!
-----------------------------------------------
sub-028 Done!
-----------------------------------------------
sub-029 

In [ ]:

for sub in sub_list:
 
    new_folder_path = os.path.join(f'/Volumes/Li/task-debate/braindata/{denoised}', 'parcel data', tmp, sub)
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)
        
    for run in run_list:

        run_file = f'/Volumes/Li/task-debate/braindata/{denoised}/denoised/{sub}/{sub}_task-debate_run-{run}_denoised_smooth6mm_space-MNI152NLin2009cAsym_res-2_bold.nii.gz'
        run_image = nib.load(run_file)
        
        masker = input_data.NiftiLabelsMasker(labels_img = Schaefer_mask_file, standardize=True)
        
        parcellations_time_series = masker.fit_transform(run_image)
        
        parcellations_time_series_df = pd.DataFrame(parcellations_time_series) # , columns=roi_label
        
        
        csv_file_path = f'/Volumes/Li/task-debate/braindata/{denoised}/parcel data/{tmp}/{sub}/{sub}_run-{run}_task-health_{tmp}_time-series.csv'
        parcellations_time_series_df.to_csv(csv_file_path, index=False)

    print(f'{sub} Done!')



# combine 6 runs

In [ ]:
new_folder_path = os.path.join('/Volumes/Li/task-debate/braindata', denoised, 'parcel data','Schaefer 200 combine 6 runs')
if not os.path.exists(new_folder_path):
    os.makedirs(new_folder_path)


for sub in sub_list:
    all_data = []
    for run in run_list:
        
        csv_files = f'/Volumes/Li/task-debate/braindata/{denoised}/parcel data/Schaefer/{sub}/{sub}_run-{run}_task-health_Schaefer_time-series.csv'
        run_data = pd.read_csv(csv_files)
        all_data.append(run_data)
        
    combined_data = pd.concat(all_data, ignore_index=True)
    combined_csv_path = os.path.join(f'/Volumes/Li/task-debate/braindata/{denoised}', 'parcel data','Schaefer 200 combine 6 runs', f'{sub}_combined_time-series_Schaefer2018_200Parcels_7Networks.csv')
    combined_data.to_csv(combined_csv_path, index=False)

    print(f"{sub} Combined CSV saved Done!!!")
